<a href="https://colab.research.google.com/github/aviramberg276/scout_exp/blob/develop/players_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import libraries
import cv2
import os
import numpy as np

In [0]:
#Reading the video
vidcap = cv2.VideoCapture('/content/drive/My Drive/Colab Notebooks/data/soccer/cutvideo.mp4')
success,image = vidcap.read()

In [70]:
image.shape[1]

1280

In [0]:
# fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
# fourcc=cv2.VideoWriter_fourcc(*'mp4v') 
# fourcc = cv2.VideoWriter_fourcc(*'X264')
width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fourcc1 = cv2.VideoWriter_fourcc(*'XVID')
# fourcc1 = cv2.CV_FOURCC(*'MJPG')
out = cv2.VideoWriter('/content/drive/My Drive/Colab Notebooks/data/soccer/test_file_out1.avi', fourcc1, 20.0,  (int(width),int(height)))

In [0]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out1 = cv2.VideoWriter('/content/drive/My Drive/Colab Notebooks/data/soccer/test_file_out2.avi', fourcc, 20.0,  (int(width),int(height)))

In [0]:

count = 0
success = True
idx = 0

In [74]:
while success:
	#converting into hsv image
	hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
	#green range
	lower_green = np.array([40,40, 40])
	upper_green = np.array([70, 255, 255])
	#blue range
	lower_blue = np.array([110,50,50])
	upper_blue = np.array([130,255,255])

	#Red range
	lower_red = np.array([0,31,255])
	upper_red = np.array([176,255,255])

	#white range
	lower_white = np.array([0,0,0])
	upper_white = np.array([0,0,255])

    #Define a mask ranging from lower to uppper
	mask = cv2.inRange(hsv, lower_green, upper_green)
    #Do masking
	res = cv2.bitwise_and(image, image, mask=mask)
	#convert to hsv to gray
	res_bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
	res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)

    #Defining a kernel to do morphological operation in threshold image to 
    #get better output.
	kernel = np.ones((13,13),np.uint8)
	thresh = cv2.threshold(res_gray,127,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
	thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
	

    #find contours in threshold image     
	contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
	

	prev = 0
	font = cv2.FONT_HERSHEY_SIMPLEX

	for c in contours:
		x,y,w,h = cv2.boundingRect(c)
		
		#Detect players
		if(h>=(1.5)*w):
			if(w>15 and h>= 15):
				idx = idx+1
				player_img = image[y:y+h,x:x+w]
				player_hsv = cv2.cvtColor(player_img,cv2.COLOR_BGR2HSV)
				#If player has blue jersy
				mask1 = cv2.inRange(player_hsv, lower_blue, upper_blue)
				res1 = cv2.bitwise_and(player_img, player_img, mask=mask1)
				res1 = cv2.cvtColor(res1,cv2.COLOR_HSV2BGR)
				res1 = cv2.cvtColor(res1,cv2.COLOR_BGR2GRAY)
				nzCount = cv2.countNonZero(res1)
				#If player has red jersy
				mask2 = cv2.inRange(player_hsv, lower_red, upper_red)
				res2 = cv2.bitwise_and(player_img, player_img, mask=mask2)
				res2 = cv2.cvtColor(res2,cv2.COLOR_HSV2BGR)
				res2 = cv2.cvtColor(res2,cv2.COLOR_BGR2GRAY)
				nzCountred = cv2.countNonZero(res2)

				if(nzCount >= 20):
					#Mark blue jersy players as france
					cv2.putText(image, 'France', (x-2, y-2), font, 0.8, (255,0,0), 2, cv2.LINE_AA)
					cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
				else:
					pass
				if(nzCountred>=20):
					#Mark red jersy players as belgium
					cv2.putText(image, 'Belgium', (x-2, y-2), font, 0.8, (0,0,255), 2, cv2.LINE_AA)
					cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
				else:
					pass
		if((h>=1 and w>=1) and (h<=30 and w<=30)):
			player_img = image[y:y+h,x:x+w]
		
			player_hsv = cv2.cvtColor(player_img,cv2.COLOR_BGR2HSV)
			#white ball  detection
			mask1 = cv2.inRange(player_hsv, lower_white, upper_white)
			res1 = cv2.bitwise_and(player_img, player_img, mask=mask1)
			res1 = cv2.cvtColor(res1,cv2.COLOR_HSV2BGR)
			res1 = cv2.cvtColor(res1,cv2.COLOR_BGR2GRAY)
			nzCount = cv2.countNonZero(res1)
	

			if(nzCount >= 3):
				# detect football
				cv2.putText(image, 'football', (x-2, y-2), font, 0.8, (0,255,0), 2, cv2.LINE_AA)
				cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
				print('Read a new frame: ', success)     # save frame as JPEG file	
	out.write(image)
	out1.write(res)
	count += 1
	# cv2.imshow('Match Detection',image)
  # checking wheather we got interupt from the user the close the image.
	# cv2.imwrite("/content/frame%d.jpg" % count, res)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break
	success,image = vidcap.read()
	# break
    
vidcap.release()
out.release()
out1.release()
cv2.waitKey(5)
cv2.destroyAllWindows()

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True


In [0]:
count

25679

In [0]:
res.shape

(720, 1280, 3)

In [0]:
res

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [0]:
cap = cv2.VideoCapture('/content/drive/My Drive/Colab Notebooks/data/soccer/alexander_sorloth.mp4')   
  
# Define the codec and create VideoWriter object 
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter('/content/drive/My Drive/Colab Notebooks/data/soccer/output.avi', fourcc, 20.0, (640, 480)) 
  
# loop runs if capturing has been initialized.  
while(True): 
    # reads frames from a camera  
    # ret checks return at each frame 
    ret, frame = cap.read()  
  
    # Converts to HSV color space, OCV reads colors as BGR 
    # frame is converted to hsv 
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 
      
    # output the frame 
    out.write(hsv)  
      
    # The original input frame is shown in the window  
    # cv2.imshow('Original', frame) 
  
    # The window showing the operated video stream  
    # cv2.imshow('frame', hsv) 
  
      
    # Wait for 'a' key to stop the program  
    if cv2.waitKey(1) & 0xFF == ord('a'): 
        break
  
# Close the window / Release webcam 
cap.release() 
  
# After we release our webcam, we also release the output 
out.release()  
  
# De-allocate any associated memory usage  
cv2.destroyAllWindows() 

error: ignored

In [0]:
Sequential()model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))model.add(MaxPooling2D(pool_size=(2, 2)))model.add(Dropout(0.25))model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))model.add(MaxPooling2D(pool_size=(2, 2)))model.add(Dropout(0.25))model.add(Flatten())model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))model.add(Dense(no_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))